<a href="https://colab.research.google.com/github/giovanniunimi1/Market-Basket/blob/main/Market_Basket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #DOWNLOAD SPARK AND SPARK DEPENDENCIES (JDK AND FINDSPARK)
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

! pip install -q kaggle
#set kaggle for dataset (remove # for setting kaggle api for downloading dataset)
from google.colab import files
uploaded = files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

os.environ['KAGGLE_USERNAME'] = "giovannibuscemi"
os.environ['KAGGLE_KEY'] = "Bsc120199.gg"


!pip install pyngrok
import getpass
from pyngrok import ngrok, conf
from collections import defaultdict
import sys
import findspark
import zipfile
! kaggle datasets download -d gsimonx37/letterboxd
#! unzip letterboxd.zip -d /letterboxd
extract_to_path = "/letterboxd"
with zipfile.ZipFile("letterboxd.zip", 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.endswith('.csv'):
            zip_ref.extract(file, extract_to_path)

def map_names(row):
  return(row.id,names_map[row.name])

def get_candidate_set(frequent_itemset,k):
  candidate_set = set()
  n=len(frequent_itemset)
  for i in range(n):
    for j in range(i+1,n):
      candidate =tuple(sorted([frequent_itemset[i],frequent_itemset[j]]))
      candidate_set.add(candidate)
  return list(candidate_set)


def apriori(iterator):
  baskets = list(iterator)

  table_counts = defaultdict(int)

  for basket in baskets:
    for item in basket:
      table_counts[item] += 1

  frequent_items = [item for item, count in table_counts.items() if count >= support]

  candidate_set = get_candidate_set(frequent_items,2)

  candidate_counts = defaultdict(int)
  for basket in baskets:
    for candidate in candidate_set:
       if all(element in basket for element in candidate):
           candidate_counts[candidate] += 1
  #filter infrequent itemset
  frequent_itemset =[candidate for candidate,count in candidate_counts.items() if count >= support]
  return frequent_itemset

def count_itemset(iterator,candidate_set):
  baskets = list(iterator)
  local_count = []

  for candidate in candidate_set:
    count = 0
    for basket in baskets:
      if all(element in basket for element in candidate):
        count +=1
    local_count.append((candidate,count))
  return local_count

def count_single_items(iterator,single_items):
  baskets = list(iterator)
  local_count = defaultdict(int)

  for basket in baskets:
    for item in single_items:
      if item in basket:
        local_count[item]+=1
  return local_count.items()

def calculate_confidence(frequent_itemset_counts,single_item_counts):
  confidence = {}
  for itemset,count in frequent_itemset_counts:
      confidence[(int_map[itemset[0]],int_map[itemset[1]])] = count / single_item_counts[itemset[0]]
      confidence[(int_map[itemset[1]],int_map[itemset[0]])] = count / single_item_counts[itemset[1]]

  return confidence
#GLOBAL PARAMETER : FRACTION OF THE DATASET, SUPPORT
fraction = 0.014
support = 4

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count,collect_list
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').appName("A-priori").getOrCreate()
sc = spark.sparkContext

actors = spark.read.csv("/letterboxd/actors.csv", header=True, inferSchema=True)


#dict for names
names = actors.select("name").distinct().rdd.map(lambda row : row[0]).collect()
names_map = {name : i for i,name in enumerate(names)}
int_map = {idx :name for name,idx in names_map.items()}


all_baskets = actors.rdd.map(map_names).groupByKey().map(lambda x: tuple(x[1]))
baskets = all_baskets.sample(False,fraction)
numPartitions = baskets.getNumPartitions()

local_support = support/numPartitions +1

#MapReduce phase 1
candidate_itemset = baskets.mapPartitions(lambda partition: apriori(partition)).collect()
#eliminate duplicates
candidate_itemset = list(set(candidate_itemset))

#MapReduce phase 2
frequent_itemset = baskets.mapPartitions(lambda partition: count_itemset(partition,candidate_itemset)).reduceByKey(lambda x,y: x+y).filter(lambda row : row[1]>=local_support).collect()
#Operation for calculate Confidence
single_items = set()
for itemset,_ in frequent_itemset :
  for item in itemset:
    single_items.add(item)

single_item_counts = baskets.mapPartitions(lambda partition : count_single_items(partition,list(single_items))).reduceByKey(lambda x,y : x+y).collectAsMap()
print("Frequent itemset :")
print(frequent_itemset)

confidence = calculate_confidence(frequent_itemset,single_item_counts)
print("Confidence for frequent itemset")
print(confidence)
spark.stop()